In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [2]:
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")
info = pd.read_csv('../data/snp_info.csv', index_col=0)

In [3]:
stocks_analysis = ['KO', 'TGT', 'PFE', 'MSFT', 'CVX', 'DVN', 'DAL', 'JPM', 'PEP', 'AAPL']

In [4]:
stocks = pd.read_csv('stocks_by_cluster.csv')
stocks

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8
0,PEP,TGT,ABT,MSFT,CVX,DVN,DAL,JPM
1,KO,TJX,PFE,AMAT,SLB,MRO,AAL,BAC
2,PG,WBA,MDT,AAPL,COP,KMI,UAL,MS
3,PM,WMT,ABBV,INTC,HAL,WMB,UNP,C
4,MDLZ,CVS,JNJ,NFLX,XOM,MGM,FAST,KEY
5,MO,M,BSX,NVDA,CAT,HPE,IPG,SCHW
6,NWL,KR,GILD,ATVI,RF,DISCA,GM,HBAN


In [5]:
idx_dict = {}
for i in range(len(stocks.index)):
    for j in range(len(stocks.columns)):
        stock = stocks.iloc[i,j]
        idx_dict[stock] = (i,j)

---------
Get Samples and Labels
---------

In [6]:
samples = pd.read_csv('samples_cluster.csv', index_col=0, header=[0,1,2], parse_dates=True, date_parser=mydateparser)
samples.head()

Cluster     cluster_1                                                       \
Asset         asset_1                                                        
Metrics         Close        RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                                         
2016-01-22  95.849998  46.758102  39.393925 -60.606075 -0.449477 -0.355034   
2016-01-25  95.940002  47.496642  40.757622 -59.242378 -0.390672 -0.362422   
2016-01-26  96.320000  50.819987  47.158198 -52.841802 -0.318094 -0.353299   
2016-01-27  95.940002  47.360939  46.945914 -53.054086 -0.282311 -0.338775   
2016-01-28  97.449997  59.881188  78.066898 -21.933102 -0.156237 -0.301598   

Cluster                                                                  \
Asset                                                asset_2              
Metrics    Price Rate Of Change On Balance Volume      Close        RSI   
Date                                                                      
2016-01-22            -0.013990           3217800  42.060001  55.344782   
2016-01-25            -0.015394           7624800  42.160000  57.575863   
2016-01-26            -0.013317          11360000  42.080002  55.037601   
2016-01-27            -0.005803           5564900  42.090000  55.321647   
2016-01-28             0.017117          10734800  42.570000  66.903600   

Cluster           ...                   cluster_8                            \
Asset             ...                     asset_6                   asset_7   
Metrics           ...        Price Rate Of Change On Balance Volume   Close   
Date              ...                                                         
2016-01-22        ...                   -0.109310        -121219600    9.15   
2016-01-25        ...                   -0.124217        -136501300    8.80   
2016-01-26        ...                   -0.129617        -124863700    8.05   
2016-01-27        ...                   -0.088777        -135186600    8.26   
2016-01-28        ...                   -0.117354        -147285700    8.48   

Cluster                                                          \
Asset                                                             
Metrics           RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                              
2016-01-22  20.659193  25.892845 -74.107155 -0.166997 -0.107536   
2016-01-25  15.455201  10.267876 -89.732124 -0.194758 -0.125617   
2016-01-26   9.523649   7.801429 -92.198571 -0.258863 -0.153038   
2016-01-27  19.504577  16.475110 -83.524890 -0.291544 -0.181377   
2016-01-28  28.975531  25.291817 -74.708183 -0.299122 -0.205358   

Cluster                                            
Asset                                              
Metrics    Price Rate Of Change On Balance Volume  
Date                                               
2016-01-22            -0.089552         -59805200  
2016-01-25            -0.128713        -131099000  
2016-01-26            -0.209234        -219455100  
2016-01-27            -0.152820        -182006200  
2016-01-28            -0.139086        -154026100  

[5 rows x 448 columns]

In [7]:
labels = pd.read_csv('labels_cluster.csv', index_col='Date', parse_dates=True, date_parser=mydateparser)
labels.head()

,PEP,KO,PG,PM,MDLZ,MO,NWL,TGT,TJX,WBA,...,FAST,IPG,GM,JPM,BAC,MS,C,KEY,SCHW,HBAN
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-25,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2016-01-26,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-01-27,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-01-28,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------------
Tensor Ensemble Learning
------------------------------------

In [8]:
from GRTEL.decomposition import GLTD
from GRTEL.utils import downturn_confidence, print_scores, print_1_percentage, confusion_matrix_metrics
from GRTEL.classification import GRTEL

from hottbox.core import Tensor, TensorTKD
from hottbox.pdtools import pd_to_tensor
from hottbox.algorithms.decomposition import HOSVD, HOOI
from hottbox.utils.generation import residual_tensor
from hottbox.algorithms.classification import TelVI

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [9]:
X = []
for i in range(len(samples)):
    X_t = samples.iloc[i].reorder_levels(['Metrics', 'Asset', 'Cluster']) #Make 'Cluster' the third mode of the tensor
    X.append(pd_to_tensor(X_t))

y = np.array(labels)

print(X[0], '\n\n', y[0])

This tensor is of order 3 and consists of 448 elements.
Sizes and names of its modes are (8, 7, 8) and ['Metrics', 'Asset', 'Cluster'] respectively. 

 [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0.]


In [10]:
# Represent each sample in Tucker form and store it in a list
algo = HOOI()
# algo = GLTD()
rank = (4,5,4)
X_tk = [algo.decompose(sample, rank=rank) for sample in X]


# Split into train and test set
# X_train, X_test, y_train, y_test = train_test_split(X_tk, y, test_size=0.25, random_state=42)

test_size = 0.25
k = int(len(X_tk) * (1. - test_size))

X_train, X_test = X_tk[:k], X_tk[k:]
y_train, y_test = y[:k], y[k:]

/Users/atayardimci/opt/anaconda3/envs/GR-TEL/lib/python3.6/site-packages/hottbox/algorithms/decomposition/base.py:85: RuntimeWarning: invalid value encountered in sqrt
  S = np.sqrt(S)


In [11]:
# Initialise classifier
R = np.sum(rank) # number of base classifiers required per class
n_classes = 1 if y.ndim == 1 else y.shape[1]

base_clfs = []
for _ in range(n_classes):
    base_clfs.append([DecisionTreeClassifier() for _ in range(R)])

grtel = GRTEL(base_clfs=base_clfs,
              n_classes=n_classes,
              probability=True,
              verbose=False)


# Train classifer
grtel.fit(X_train, y_train)

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 


In [12]:
### Classifier performance results
# train accuracy
train_score = grtel.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = grtel.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = grtel.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=stocks_analysis, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in stocks_analysis:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL
train acc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
test acc,0.557377,0.579235,0.590164,0.601093,0.530055,0.502732,0.530055,0.508197,0.584699,0.557377
precision,0.581699,0.571429,0.691057,0.639706,0.523438,0.461538,0.617978,0.443609,0.580645,0.554622
recall,0.839623,0.824742,0.696721,0.783784,0.728261,0.348837,0.514019,0.786667,0.891089,0.702128
downturn_precision,0.433333,0.604651,0.383333,0.489362,0.545455,0.525424,0.446809,0.680000,0.607143,0.562500
specificity,0.168831,0.302326,0.377049,0.319444,0.329670,0.639175,0.552632,0.314815,0.207317,0.404494


In [13]:
#Scores
score = grtel.score(X_train, y_train)
print("\nClassification accuracy (Train):")
print_scores(score); print()

score = grtel.score(X_test, y_test)
print("Classification accuracy (Test):")
print_scores(score); print()

print("Percentage of 1s (Test):")
print_1_percentage(y_test, n_classes); print()


Classification accuracy (Train):
[100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%]

Classification accuracy (Test):
[58.47%, 55.74%, 44.81%, 54.10%, 44.26%, 51.37%, 48.09%, 57.92%, 55.74%, 50.27%, 57.38%, 54.10%, 50.27%, 46.45%, 54.64%, 59.02%, 43.17%, 50.27%, 55.74%, 57.38%, 45.90%, 60.11%, 43.72%, 55.74%, 48.63%, 51.37%, 50.82%, 42.08%, 53.01%, 45.90%, 54.10%, 44.26%, 48.09%, 51.91%, 37.70%, 50.27%, 49.18%, 52.46%, 54.10%, 38.80%, 51.37%, 50.82%, 53.01%, 48.63%, 56.28%, 55.74%, 44.81%, 49.73%, 56.83%, 50.82%, 49.18%, 46.99%, 49.73%, 37.

-------
Grid Search
------

In [14]:
max_features = ['auto', 'sqrt', None, 'log2']
max_depth = list(range(10, 70, 10))
max_depth.append(None)
min_samples_split = [2, 5, 10, 20, 30]
min_samples_leaf = [1, 3, 5, 7, 12, 14]

search_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

search_params = [search_grid for _ in range(R)]

print("\tPerforming grid search for each base classifer and for each class")
grtel.grid_search(X_train, y_train, search_params)

print("\tTrain base classifiers with optimal hyperparameters")
grtel.fit(X_train, y_train); print()

	Performing grid search for each base classifer and for each class
0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 
	Train base classifiers with optimal hyperparameters
0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 



In [15]:
### Classifier performance results
# train accuracy
train_score = grtel.score(X_train, y_train)
train_accuracy_df = pd.DataFrame(np.array(train_score).reshape(stocks.shape[1], stocks.shape[0])).T
train_accuracy_df.columns = stocks.columns

# test accuracy
test_score = grtel.score(X_test, y_test)
test_accuracy_df = pd.DataFrame(np.array(test_score).reshape(stocks.shape[1], stocks.shape[0])).T
test_accuracy_df.columns = stocks.columns

# confusion matrices
conf_matrices = grtel.confusion_matrices(X_test, y_test)
conf_matrices = np.array(conf_matrices).reshape(stocks.shape[1], stocks.shape[0], 2, 2)



## Print performance results
results = pd.DataFrame(columns=stocks_analysis, index=['train acc', 'test acc', 'precision', 'recall', 'downturn_precision', 'specificity'], dtype=float)
for stock in stocks_analysis:
    idx = idx_dict[stock]
    results[stock]['train acc'] = train_accuracy_df.iloc[idx]
    results[stock]['test acc'] = test_accuracy_df.iloc[idx]
    
    conf_matrix = conf_matrices[idx[1],idx[0]]
    accuracy, precision, recall, specificity, downturn_precision = confusion_matrix_metrics(conf_matrix)
    
    results[stock]['precision'] = precision
    results[stock]['recall'] = recall
    results[stock]['downturn_precision'] = downturn_precision
    results[stock]['specificity'] = specificity
    
results

,KO,TGT,PFE,MSFT,CVX,DVN,DAL,JPM,PEP,AAPL
train acc,0.998179,0.998179,0.998179,1.000000,0.998179,0.996357,1.000000,0.970856,1.000000,0.998179
test acc,0.530055,0.497268,0.601093,0.540984,0.590164,0.486339,0.535519,0.437158,0.579235,0.568306
precision,0.584000,0.526718,0.693548,0.581560,0.551948,0.378378,0.727273,0.383459,0.591241,0.560811
recall,0.688679,0.711340,0.704918,0.738739,0.923913,0.162791,0.523364,0.680000,0.801980,0.882979
downturn_precision,0.431034,0.461538,0.389831,0.309524,0.758621,0.506849,0.518868,0.520000,0.565217,0.685714
specificity,0.324675,0.279070,0.377049,0.180556,0.241758,0.762887,0.723684,0.240741,0.317073,0.269663


In [16]:
score = grtel.score(X_train, y_train)
print("Classification accuracy (Train):")
print_scores(score); print()

score = grtel.score(X_test, y_test)
print("Classification accuracy (Test):")
print_scores(score); print()

print("Percentage of 1s (Test):")
print_1_percentage(y_test, n_classes); print()

Classification accuracy (Train):
[100.00%, 99.82%, 99.09%, 99.45%, 100.00%, 99.27%, 100.00%, 99.82%, 96.72%, 100.00%, 98.54%, 100.00%, 100.00%, 100.00%, 100.00%, 99.82%, 100.00%, 100.00%, 100.00%, 100.00%, 99.82%, 100.00%, 99.82%, 99.82%, 100.00%, 100.00%, 99.64%, 100.00%, 99.82%, 100.00%, 99.82%, 99.82%, 99.27%, 99.82%, 99.82%, 99.64%, 100.00%, 99.82%, 99.45%, 100.00%, 97.09%, 99.64%, 100.00%, 100.00%, 100.00%, 99.09%, 100.00%, 100.00%, 100.00%, 97.09%, 99.82%, 99.82%, 100.00%, 100.00%, 99.82%, 99.64%]

Classification accuracy (Test):
[57.92%, 53.01%, 46.45%, 48.09%, 58.47%, 52.46%, 42.08%, 49.73%, 62.84%, 53.55%, 51.91%, 43.17%, 42.08%, 52.46%, 49.18%, 60.11%, 40.44%, 42.62%, 57.38%, 66.67%, 49.18%, 54.10%, 39.89%, 56.83%, 52.46%, 44.26%, 41.53%, 43.72%, 59.02%, 40.98%, 53.55%, 49.73%, 54.64%, 54.10%, 45.90%, 48.63%, 54.64%, 43.17%, 50.27%, 41.53%, 45.36%, 50.82%, 53.55%, 57.92%, 55.74%, 49.18%, 46.99%, 45.36%, 63.93%, 43.72%, 37.70%, 44.26%, 54.64%, 59.02%, 48.63%, 41.53%]

Percenta